# InitialSetup

In [ ]:
import os
import shutil

if not os.path.exists('gen_data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if os.path.exists('OUTPUT/'):
    shutil.rmtree('OUTPUT/')
    
os.makedirs('OUTPUT/')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, AveragePooling2D, MaxPooling2D, Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Loading dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)


test_gen = ImageDataGenerator(rescale=1./255)

training_data = train_gen.flow_from_directory('gen_data_set/train', target_size=(265,256), 
                                                 batch_size=32, class_mode='binary')

test_data = test_gen.flow_from_directory('gen_data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

# Training

In [ ]:
validation_steps = len(test_data)

# Training model from scratch

In [ ]:
classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
steps_per_epoch = len(training_data)

classifier.fit(training_data, epochs=90, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=test_data, validation_steps=validation_steps)

In [ ]:
classifier.save("OUTPUT/ScratchModel.h5")

In [ ]:
_, accuracy = classifier.evaluate(test_data, steps = validation_steps)

In [ ]:
print('ScratchModel accuracy :',accuracy)

# Using pre-trained Models

In [ ]:
class Wrapper(tf.keras.Model):
    def __init__(self, base_model):
        super(Wrapper, self).__init__()
        
        self.base_model = base_model
        self.average_pooling_layer = AveragePooling2D(name="polling")
        self.flatten = Flatten(name="flatten")
        self.dense = Dense(64, activation="relu")
        self.dropout = Dropout(0.5)
        self.output_layer = Dense(2, activation="softmax")
        
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.average_pooling_layer(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dropout(x)
        output = self.output_layer(x)
        return output

In [ ]:
base_learning_rate = 0.0001
EPOCHS = 10

## ResNet

In [ ]:
from tensorflow.keras.applications import ResNet152V2

In [ ]:
res_net = ResNet152V2(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [ ]:
res_net.trainable = False
res_net_classifier = Wrapper(res_net)
res_net_classifier.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
res_net_classifier.fit(training_data, epochs=EPOCHS, validation_data=test_data, validation_steps=validation_steps)